# DSLab Homework3 - Uncovering World Events using Twitter Hashtags

## ... and learning about Spark `DataFrames` along the way

In this notebook, we will use temporal information about Twitter hashtags to discover trending topics and potentially uncover world events as they occurred. 

## Hand-in Instructions:

- __Due: 27.04.2021 23:59:59 CET__
- your project must be private
- `git push` your final verion to the master branch of your group's Renku repository before the due date
- check if `Dockerfile`, `environment.yml` and `requirements.txt` are properly written
- add necessary comments and discussion to make your codes readable

## Hashtags

The idea here is that when an event is happening and people are having a conversation about it on Twitter, a set of uniform hashtags that represent the event spontaneously evolves. Twitter users then use those hashtags to communicate with one another. Some hashtags, like `#RT` for "retweet" or just `#retweet` are used frequently and don't tell us much about what is going on. But a sudden appearance of a hashtag like `#oscars` probably indicates that the oscars are underway. For a particularly cool example of this type of analysis, check out [this blog post about earthquake detection using Twitter data](https://blog.twitter.com/official/en_us/a/2015/usgs-twitter-data-earthquake-detection.html) (although they search the text and not necessarily hashtags).

## Initialize the environment

In [ ]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-homework3", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

Send `username` to Saprk kernel, which will frist start the Spark application if there is no active session.

In [ ]:
%%send_to_spark -i username -t str -n username

In [ ]:
print('We are using Spark %s' % spark.version)

In [ ]:
%%help

## PART I: Set up (5 points)

The twitter stream data is downloaded from [Archive Team: The Twitter Stream Grab](https://archive.org/details/twitterstream), which is a collection of a random sample of all tweets. We have parsed the stream data and prepared the twitter hashtag data of __2020__, a very special and different year in many ways. Let's see if we can see any trends about all these events of 2020 in the Twitter data. 

<div style="font-size: 100%" class="alert alert-block alert-danger">
<b>Disclaimer</b>
<br>
This dataset contains unfiltered data from Twitter. As such, you may be exposed to tweets/hashtags containing vulgarities, references to sexual acts, drug usage, etc.
</div>

### a) Load data - 1/10

Load the **orc** data from `/data/twitter/orc/hashtags/` into a Spark dataframe using the appropriate `SparkSession` method. 

Look at the first few rows of the dataset - note the timestamp and its units!

In [ ]:
# TODO
# df = spark.<your_code>
# Load the data
df = spark.read.orc("/data/twitter/orc/hashtags")

In [ ]:
# Show the schema
df.printSchema()

In [ ]:
# Show the first 5 line
df.show(n=5, truncate=False, vertical=False)

<div style="font-size: 100%" class="alert alert-block alert-info">
    <b>Cluster Usage:</b> As there are many of you working with the cluster, we encourage you to
    <ul>
        <li>prototype your queries on small data samples before running them on whole datasets</li>
        <li>save your intermediate results in your own directory at hdfs <b>"/user/&lt;your-gaspar-id&gt;/"</b></li>
    </ul>
</div>

For example:

```python
    # create a subset of original dataset
    df_sample = df.sample(0.01)
    
    # save as orc
    df_sample.write.orc('/user/%s/sample.orc' % username, mode='overwrite')

```

### b) Functions - 2/10

In [ ]:
import pyspark.sql.functions as F

__User-defined functions__

A neat trick of spark dataframes is that you can essentially use something very much like an RDD `map` method but without switching to the RDD. If you are familiar with database languages, this works very much like e.g. a user-defined function in SQL. 

So, for example, if we wanted to make a user-defined python function that returns the hashtags in lowercase, we could do something like this:

In [ ]:
@F.udf
def lowercase(text):
    """Convert text to lowercase"""
    return text.lower()

The `@F.udf` is a "decorator" -- this is really handy python syntactic sugar and in this case is equivalent to:

```python
def lowercase(text):
    return text.lower()
    
lowercase = F.udf(lowercase)
```

It basically takes our function and adds to its functionality. In this case, it registers our function as a pyspark dataframe user-defined function (UDF).

Using these UDFs is very straightforward and analogous to other Spark dataframe operations. For example:

In [ ]:
df.select(lowercase(df.hashtag)).show(n=5)

__Built-in functions__

Using a framework like Spark is all about understanding the ins and outs of how it functions and knowing what it offers. One of the cool things about the dataframe API is that many functions are already defined for you (turning strings into lowercase being one of them). Find the [Spark python API documentation](https://spark.apache.org/docs/2.3.2/api/python/index.html). Look for the `sql` section and find the listing of `sql.functions`. Repeat the above (turning hashtags into lowercase) but use the built-in function.

In [ ]:
# TODO
# df.<your_code>
from pyspark.sql.functions import lower
# turning hashtags into lowercase
df_lowercase = df.withColumn("hashtag", lower(df.hashtag))
df_lowercase.show(5)

We'll work with a combination of these built-in functions and user-defined functions for the remainder of this homework. 

Note that the functions can be combined. Consider the following dataframe and its transformation:

In [ ]:
from pyspark.sql import Row

# create a sample dataframe with one column "degrees" going from 0 to 180
test_df = spark.createDataFrame(spark.sparkContext.range(180).map(lambda x: Row(degrees=x)), ['degrees'])

# define a function "sin_rad" that first converts degrees to radians and then takes the sine using built-in functions
sin_rad = F.sin(F.radians(test_df.degrees))

# show the result
test_df.select(sin_rad).show()

### c) Tweets in english - 2/10

- Create `english_df` with only english-language tweets. 
- Turn hashtags into lowercase.
- Convert the timestamp to a more readable format and name the new column as `date`.
- Sort the table in chronological order. 

Your `english_df` should look something like this:

```
+-----------+----+-----------+-------------------+
|timestamp_s|lang|    hashtag|               date|
+-----------+----+-----------+-------------------+
| 1577862000|  en| spurfamily|2020-01-01 08:00:00|
| 1577862000|  en|newyear2020|2020-01-01 08:00:00|
| 1577862000|  en|     master|2020-01-01 08:00:00|
| 1577862000|  en|  spurrific|2020-01-01 08:00:00|
| 1577862000|  en|     master|2020-01-01 08:00:00|
+-----------+----+-----------+-------------------+
```

__Note:__ 
- The hashtags may not be in english.
- [pyspark.sql.functions](https://spark.apache.org/docs/2.3.2/api/python/pyspark.sql.html#module-pyspark.sql.functions)

In [ ]:
# TODO
# english_df = df.<your_code>
# select the data
english_df = df_lowercase.filter(df_lowercase.lang == 'en')
# order the data
english_df = english_df.withColumn('date', F.from_unixtime('timestamp_s','yyyy-MM-dd hh:mm:ss')).orderBy('date')
english_df = english_df.drop(english_df.id)

In [ ]:
# show the english data
english_df.show(n=5)

## PART II: Twitter hashtag trends (30 points)

In this section we will try to do a slightly more complicated analysis of the tweets. Our goal is to get an idea of tweet frequency as a function of time for certain hashtags. 

Have a look [here](http://spark.apache.org/docs/2.3.2/api/python/pyspark.sql.html#module-pyspark.sql.functions) to see the whole list of custom dataframe functions - you will need to use them to complete the next set of TODO items.

### a) Top hashtags - 1/30

We used `groupBy` already in the previous notebooks, but here we will take more advantage of its features. 

One important thing to note is that unlike other RDD or DataFrame transformations, the `groupBy` does not return another DataFrame, but a `GroupedData` object instead, with its own methods. These methods allow you to do various transformations and aggregations on the data of the grouped rows. 

Conceptually the procedure is a lot like this:

![groupby](https://i.stack.imgur.com/sgCn1.jpg)

The column that is used for the `groupBy` is the `key` - once we have the values of a particular key all together, we can use various aggregation functions on them to generate a transformed dataset. In this example, the aggregation function is a simple `sum`. In the simple procedure below, the `key` will be the hashtag.


Use `groupBy`, calculate the top 5 most common hashtags in the whole english-language dataset.

This should be your result:

```
+-----------------+-------+
|          hashtag|  count|
+-----------------+-------+
|              bts|1200196|
|          endsars|1019280|
|          covid19| 717238|
|            방탄소년단| 488160|
|sarkaruvaaripaata| 480124|
+-----------------+-------+
```

In [ ]:
# TODO
# english_df.<your_code>
# calculate the top 5 most common hashtags in the whole english-language dataset.
hashtag_count = english_df.groupBy('hashtag')\
                          .agg(F.count('hashtag').alias('count'))\
                          .sort(F.desc(('count')))
hashtag_count.show(5)

### b) Daily hashtags - 2/50

Now, let's see how we can start to organize the tweets by their timestamps. Remember, our goal is to uncover trending topics on a timescale of a few days. A much needed column then is simply `day`. Spark provides us with some handy built-in dataframe functions that are made for transforming date and time fields.

- Create a dataframe called `daily_hashtag` that includes the columns `month`, `week`, `day` and `hashtag`. 
- Use the `english_df` you made above to start, and make sure you find the appropriate spark dataframe functions to make your life easier. For example, to convert the date string into day-of-year, you can use the built-in [dayofyear](http://spark.apache.org/docs/2.3.2/api/python/pyspark.sql.html#pyspark.sql.functions.dayofyear) function. 
- For the simplicity of following analysis, filter only tweets of 2020.
- Show the result.

Try to match this view:

```
+-----+----+---+-----------+
|month|week|day|    hashtag|
+-----+----+---+-----------+
|    1|   1|  1| spurfamily|
|    1|   1|  1|newyear2020|
|    1|   1|  1|     master|
|    1|   1|  1|  spurrific|
|    1|   1|  1|     master|
+-----+----+---+-----------+
```

In [ ]:
# TODO
# daily_hashtag = english_df.<your_code>
# Drop unnecessary columns
daily_hashtag = english_df.drop(english_df.timestamp_s).drop(english_df.lang)

# Create new dataframe with month, week of the year and day of the year
daily_hashtag = daily_hashtag.withColumn('month', F.month(english_df.date))\
                             .withColumn('week', F.weekofyear(english_df.date))\
                             .withColumn('day', F.dayofyear(english_df.date))\
                            .withColumn('year', F.year(english_df.date))
daily_hashtag = daily_hashtag.filter('year == "2020"') ## FILTER IS ADDED HERE
daily_hashtag = daily_hashtag.select(F.col('month'), F.col('week'), 
                                     F.col('day'), F.col('hashtag'))

In [ ]:
# show the result
daily_hashtag.show(n=5)

### c) Daily counts - 2/50

Now we want to calculate the number of times a hashtag is used per day based on the dataframe `daily_hashtag`. Sort in descending order of daily counts and show the result. Call the resulting dataframe `day_counts`.

Your output should look like this:

```
+---+----------------------+----+------+
|day|hashtag               |week|count |
+---+----------------------+----+------+
|229|pawankalyanbirthdaycdp|33  |202241|
|222|hbdmaheshbabu         |32  |195718|
|228|pawankalyanbirthdaycdp|33  |152037|
|357|100freeiphone12       |52  |122068|
|221|hbdmaheshbabu         |32  |120401|
+---+----------------------+----+------+
```

<div class="alert alert-info">
<p>Make sure you use <b>cache()</b> when you create <b>day_counts</b> because we will need it in the steps that follow!</p>
</div>

In [ ]:
# TODO
# day_counts = daily_hashtag.<your_code>
# calculate the number of times a hashtag is used per day and sort in descending order of daily counts.
day_counts = daily_hashtag.groupBy('day','hashtag','week')\
                       .agg(F.count('hashtag').alias('count'))\
                       .sort(F.desc(('count'))).cache() #Cache added

In [ ]:
# show the result
day_counts.show(n=5, truncate=False)

### d) Weekly average - 2/50

To get an idea of which hashtags stay popular for several days, calculate the average number of daily occurences for each week. Sort in descending order and show the top 20.

__Note:__
- Use the `week` column we created above.
- Calculate the weekly average using `F.mean(...)`.

In [ ]:
# TODO
# day_counts.<your_code>
# calculate the average number of daily occurences for each week and sort in descending order and show the top 20.
week_avg = day_counts.groupBy('week', 'hashtag')\
                   .agg(F.mean('count').alias('weekly_occur_avg'))\
                   .sort(F.desc(('weekly_occur_avg')))
week_avg.show(20)

### e) Ranking - 3/20

Window functions are another awesome feature of dataframes. They allow users to accomplish complex tasks using very concise and simple code. 

Above we computed just the hashtag that had the most occurrences on *any* day. Now lets say we want to know the top tweets for *each* day.  

This is a non-trivial thing to compute and requires "windowing" our data. I recommend reading this [window functions article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) to get acquainted with the idea. You can think of a window function as a fine-grained and more flexible `groupBy`. 

There are two things we need to define to use window functions:

1. the "window" to use, based on which columns (partitioning) and how the rows should be ordered 
2. the computation to carry out for each windowed group, e.g. a max, an average etc.

Lets see how this works by example. We will define a window function, `daily_window` that will partition data based on the `day` column. Within each window, the rows will be ordered by the daily hashtag count that we computed above. Finally, we will use the rank function **over** this window to give us the ranking of top tweets. 

In the end, this is a fairly complicated operation achieved in just a few lines of code! (can you think of how to do this with an RDD??)

In [ ]:
from pyspark.sql import Window

First, we specify the window function and the ordering:

In [ ]:
daily_window = Window.partitionBy('day').orderBy(F.desc('count'))

The above window function says that we should window the data on the `day` column and order it by count. 

Now we need to define what we want to compute on the windowed data. We will start by just calculating the daily ranking of hashtags, so we can use the helpful built-in `F.rank()` and sort:

In [ ]:
# calculating the daily ranking of hashtags
daily_rank = F.rank() \
              .over(daily_window) \
              .alias('rank')

Now compute the top five hashtags for each day in our data:

In [ ]:
# TODO
# day_counts.<your_code>
# Create new dataframe with the top 5 hashtags for each day
top_five = day_counts.select(F.col('day'), F.col('count'),
                             F.col('hashtag'), daily_rank)# minor modification
top_five = top_five.filter(top_five.rank < 6).orderBy('day', 'rank')

top_five.show(10)

### f) Rolling sum - 5/30

With window functions, you can also calculate the statistics of a rolling window. 

In this question, construct a 7-day rolling window (including the day and 6 days before) to calculate the rolling sum of the daily occurences for each hashtag.

Your results should be like:
- For the hashtag `covid19`:

```
+---+----+-----+-------+-----------+
|day|week|count|hashtag|rolling_sum|
+---+----+-----+-------+-----------+
| 42|   7|   85|covid19|         85|
| 43|   7|   94|covid19|        179|
| 45|   7|  192|covid19|        371|
| 46|   7|   97|covid19|        468|
| 47|   7|  168|covid19|        636|
| 48|   8|  317|covid19|        953|
| 49|   8|  116|covid19|        984|
| 51|   8|  234|covid19|       1124|
| 52|   8|  197|covid19|       1129|
| 53|   8|  369|covid19|       1401|
+---+----+-----+-------+-----------+
```

- For the hashtag `bts`:

```
+---+----+-----+-------+-----------+
|day|week|count|hashtag|rolling_sum|
+---+----+-----+-------+-----------+
|  1|   1| 2522|    bts|       2522|
|  2|   1| 1341|    bts|       3863|
|  3|   1|  471|    bts|       4334|
|  4|   1|  763|    bts|       5097|
|  5|   1| 2144|    bts|       7241|
|  6|   2| 1394|    bts|       8635|
|  7|   2| 1673|    bts|      10308|
|  8|   2| 5694|    bts|      13480|
|  9|   2| 5942|    bts|      18081|
| 10|   2| 5392|    bts|      23002|
+---+----+-----+-------+-----------+
```

In [ ]:
# TODO
# calculate the rolling sum of the daily occurences for each hashtag.
rolling_sum_window = Window.partitionBy('hashtag').orderBy(F.asc('day')).rangeBetween(-6,0)
# rs_counts = day_counts.<your_code>
rs_counts = day_counts.select(F.col('day'), F.col('week'), F.col('count'),
                             F.col('hashtag'), F.sum('count')\
                     .over(rolling_sum_window).alias('rolling_sum'))

In [ ]:
# show the hashtag covid19
rs_counts.filter('hashtag == "covid19"').show(n=10)

In [ ]:
# show the hashtag bts
rs_counts.filter('hashtag == "bts"').show(n=10)

### g) DIY - 15/20

Use window functions (or other techniques!) to produce lists of top few trending tweets for each week. What's a __"trending"__ tweet? Something that seems to be __suddenly growing very rapidly in popularity__. 

You should be able to identify, for example, Oscars-related hashtags in week 7 when [the 92nd Academy Awards ceremony took place](https://www.oscars.org/oscars/ceremonies/2020), COVID-related hashtags in week 11 when [WHO declared COVID-19 a pandemic](https://www.who.int/director-general/speeches/detail/who-director-general-s-opening-remarks-at-the-media-briefing-on-covid-19---11-march-2020), and other events like the movement of Black Life Matters in late May, the United States presidential elections, the 2020 American Music Awards, etc.

The final listing should be clear and concise and the flow of your analysis should be easy to follow. If you make an implementation that is not immediately obvious, make sure you provide comments either in markdown cells or in comments in the code itself.

In [ ]:
# TODO

# Create new dataframe with hashtag counts per week 
week_counts = daily_hashtag.cache()
week_counts = week_counts.groupBy('week','hashtag')\
                         .agg(F.count('hashtag').alias('counts'))\
                         .sort('hashtag','week')


# Add new column to dataframe with value from previous week for each hashtag
# The first-week data for each hashtag gets the previous value set to null
counts_window = Window.partitionBy('hashtag').orderBy(F.asc('week'))
week_counts = week_counts.withColumn('prev_value', 
                                     F.lag(week_counts.counts)\
                                              .over(counts_window))

# Calculate the difference for each week compared to previous week
# The first week that a hashtag shows, the difference is set to its value 
week_counts = week_counts.withColumn('diff', 
                                     F.when(F.isnull(week_counts.counts - week_counts.prev_value), 
                                                    week_counts.counts)
                                              .otherwise((week_counts.counts - week_counts.prev_value)))

# Rank the hashtags based on 'diff' column
ranks_window = Window.partitionBy('week').orderBy(F.desc('diff'))
week_ranks = week_counts.select(F.col('week'), F.col('hashtag'),
                                F.col('counts'), F.col('diff'),
                                F.rank().over(ranks_window).alias('rank'))

# Filter out the top 5 ranks for each week
top_week = week_ranks.filter(week_ranks.rank < 6).orderBy('week', 'rank')

In [ ]:
top_week.show(10)

We can see the top five hashtags in the first two weeks. Bts appeared in both two weeks.

In [ ]:
top_week.filter(top_week.week==7).show()

we can see the oscars is the rank 1 hashtag in week 7. We can speculate that the oscar is being held in the week7.

In [ ]:
top_week.filter(top_week.week==11).show()

we can see the coronavirus and covid19 are the rank1 and rank2 hashtag in week 11. We can speculate that the covid19 large-scale outbreak all over the world in the week11.

In [ ]:
top_week.filter(top_week.hashtag.contains('bts')).show()
top_week.filter(top_week.hashtag.contains('covid')).show()
top_week.filter(top_week.hashtag.contains('coronavirus')).show()

We can see that bts and covid19 are hot hashtag that last the whole year.

## PART III: Hashtag clustering (25 points)

### a) Feature vector - 3/25

- Create a dataframe `daily_hashtag_matrix` that consists of hashtags as rows and daily counts as columns (hint: use `groupBy` and methods of `GroupedData`). Each row of the matrix represents the time series of daily counts of one hashtag. Cache the result.

- Create the feature vector which consists of daily counts using the [`VectorAssembler`](https://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler) from the Spark ML library. Cache the result.

In [ ]:
# TODO
# daily_hashtag_matrix = ...
# df_vector = ...
from pyspark.ml.feature import VectorAssembler
daily_hashtag_matrix = daily_hashtag.groupBy('hashtag').pivot('day').count().na.fill(0).drop('day').cache()
daily_hashtag_matrix.show(10)
df_count_cols = ([c for c in daily_hashtag_matrix.columns if c!="hashtag"])
vecAssembler = VectorAssembler(inputCols=df_count_cols, outputCol="features")
df_vector = vecAssembler.transform(daily_hashtag_matrix)\
                        .select("hashtag","features").cache()
df_vector.show(10)


### b) Visualization - 2/25

Visualize the time sereis you just created. 

- Select a few interesting hashtags you identified above. `isin` method of DataFrame columns might be useful.
- Convert the subset DataFrame as pandas DataFrame with the method `toPandas`.
- Plot the time series for the chosen hashtags with matplotlib. To visualize in this PySpark kernel, you need the magic `%matplot plt`. For details, please check [here](https://raw.githubusercontent.com/jupyter-incubator/sparkmagic/master/screenshots/matplotlib.png).

In [ ]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pylab as plt

plt.rcParams['figure.figsize'] = (30,8)
plt.rcParams['font.size'] = 12
plt.style.use('fivethirtyeight')

In [ ]:
# TODO
# The hashtags we identified above do not have much correlations, which may not be instrumental for explanation,
# Here, we pick up the following hashtags, including two US president election candidates (biden and trump), 
# two great events (covid and blm), two lengendary atheletes unfortunately passing away in 2020 (kobe and maradona)
interesting_list=["covid","maradona","blm","trump","biden","kobe"]
interesting_hashtags = daily_hashtag_matrix.filter(daily_hashtag_matrix.hashtag.isin(interesting_list))
interesting_hashtags.show()

In [ ]:
# converted to Pandas dataframe
interesting_hashtag_df = interesting_hashtags.toPandas().set_index('hashtag')
interesting_hashtag_df.head()

In [ ]:
# plot the popularity of the hashtags in the course of time
ax = interesting_hashtag_df.T.plot(title = 'daily occurences of certain hashtags')
ax.set_xlabel("day")
ax.set_ylabel("daily occurences")

%matplot plt

### c) KMeans clustering - 20/25

Use KMeans to cluster hashtags based on the daily count timeseries you created above. Train the model and calculate the cluster membership for all hashtags. Again, be creative and see if you can get meaningful hashtag groupings. 

Validate your results by showing certain clusters, for example, those including some interesting hashtags you identified above. Do they make sense?

Make sure you document each step of your process well so that your final notebook is easy to understand even if the result is not optimal or complete. 

__Note:__ 
- Additional data cleaning, feature engineering, deminsion reduction, etc. might be necessary to get meaningful results from the model. 
- For available methods, check [pyspark.sql.functions documentation](https://spark.apache.org/docs/2.3.2/api/python/pyspark.sql.html#module-pyspark.sql.functions), [Spark MLlib Guide](https://spark.apache.org/docs/2.3.2/ml-guide.html) and [pyspark.ml documentation](https://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html).

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Normalizer
import numpy as np

In [ ]:
# TODO
# First, we launch a mini clustering task to analyse those 6 interesting hashtags, with the number of clusters being 5
# We would like to put together those hashtags which went through a chronologically similar process in the year
# of 2020, which may indicate correlations between them. However, since events vary a lot in their popularity 
# intrinsically, we first normalize the feature vectors to bring close the events which came and left at the same time,
# but differ a lot in popularities. For example, biden and trump should be highly related. Yet, trump is much more
# popular than biden for most of the time.
interesting_vector = vecAssembler.transform(interesting_hashtags)\
                        .select("hashtag","features").cache()
interesting_vector.show()
#perform normalization using 1-norm
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
l1NormData = normalizer.transform(interesting_vector).drop("features")
l1NormData.show()

In [ ]:
#perform 5-mean clustering

kmeans = KMeans().setK(5).setFeaturesCol("normFeatures").setSeed(1)
model_interesting = kmeans.fit(l1NormData)

predicted_interesting = model_interesting.transform(l1NormData)
predicted_interesting.show()
# As can be seen from the result below. Were the six hashtags forcefully catagorized into 5 groups, trump and biden
# are in one group, which makes sense.


In [ ]:
# Next, we perform the similar workflow to the entire hashtag dataframe
# normalize everything
df_vector_norm = normalizer.transform(df_vector).drop("features").cache()
df_vector_norm.show(5)

In [ ]:
df_vector_norm = df_vector_norm.withColumnRenamed("normFeatures","features")
df_vector_norm.show(5)

In [ ]:
# selection of K

# We will try to select a proper K with the help of the 'cost' yielded by different clustering K's.
# Note, this part may take a very long time to run. To make things easier, after experiments, we adopt the following 
# strategy: We only investigate K equaling to multiples of 100 and smaller than 1000. For every K, we fit two models 
# and get the mean cost of the two models.


K_selection_num = 9
start_K = 100  
cost = np.zeros(K_selection_num)
tested_K = np.zeros(K_selection_num)

for point in range(K_selection_num):
    K = start_K + point * 100
    kmeans = KMeans().setK(K).setSeed(1)
    model1 = kmeans.fit(df_vector_norm.sample(0.01, seed = 1))
    model2 = kmeans.fit(df_vector_norm.sample(0.01, seed = 2))
    cost1 = model1.computeCost(df_vector_norm)
    cost2 = model2.computeCost(df_vector_norm)
    tested_K[point] = K
    cost[point] = np.mean([cost1, cost2])
    pass

In [ ]:
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(tested_K,cost)
ax.set_xlabel('k', fontsize = 10)
ax.set_ylabel('cost', fontsize = 10)
ax.set_title('Cost of the Kmeans algorithm w.r.t the value of K', fontsize = 12)
ax.tick_params(axis='both', which='major', labelsize=10)

# From the plot below, we can see that the cost almost levels off at about k=600. Thus, we use this value for 
# model fitting.
# Note that we can tell that the 'elbow point' of this clustering process is at around 400. However, from later analysis,
# We can tell that even clustering number of 600 is not 'fine-grained' enough. Perhaps this rough analysis of the 
# hashtags, without taking other meta information into consideration, is less effective. 
%matplot plt


In [ ]:
# Then, we fit the model and transform the dataframe
kmeans = KMeans().setK(600).setSeed(2)
model = kmeans.fit(df_vector_norm.sample(0.01, seed = 2))
transformed = model.transform(df_vector_norm)
transformed.show(10)

In [ ]:
# We then revisit the clustering process of the six interesting hashtags which we picked out.\n",
interesting_rows = transformed.filter(transformed.hashtag.isin(["covid","maradona","blm","trump","biden","kobe"]))
interesting_rows.show(10)
# But disappointedly, as is shown below, the categorization is not satisfying. We think the crudeness of clusters
# is to be blamed.

In [ ]:
# However, we did detect meaningful clustering in the imperfect result
# For example, the four hashtag related to the oscar award are brought together in the clustering process.
# P.S. the second and the third hashtags are the recipient of the Best Picture Award of Oscar
oscar_row = transformed.filter(transformed.hashtag.isin(['oscar','기생충','parasite','92ndacademyawards']))
oscar_row.show(10)

# That's all, folks!